In [1]:
# default_exp plotting.main

In [2]:
# export
import matplotlib.pyplot as plt
#from pathos.multiprocessing import ProcessingPool as Pool

In [2]:
# export 
import matplotlib.pyplot as plt
from montlake.plotting.manifolds import plot_manifold_2d, plot_manifold_3d,plot_manifold_featurespace,plot_manifold_3d_set
from montlake.plotting.plotting import plot_cosines, get_cmap,get_names, plot_cosines_cluster
from montlake.plotting.flasso import plot_reg_path_ax_lambdasearch_customcolors_norm,plot_watch_custom, plot_watch,plot_cos_boxes, plot_reg_path_ax_lambdasearch_customcolors
import dill as pickle
import pathos
from montlake.utils.utils import data_stream_custom_range, cosine_similarity
from pathos.multiprocessing import ProcessingPool as Pool
from montlake.atomgeom.features import get_features
from montlake.utils.utils import get_atoms4_full, get_index_matching, get_cosines
from montlake.utils.replicates import get_detected_values2d
from montlake.statistics.supportrecovery import get_min_min, get_mu_full_ind, get_kappa_s, get_gamma_max
import numpy as np
import itertools
import seaborn as sns
import pdb
from matplotlib.patches import Rectangle

def plot_experiment(result_file, 
                    positions,
                    d,
                    name, 
                    outdir,
                    embedding= True, 
                    ground_truth = None, 
                    colors_id_all = None, 
                    color_counts_all = None,
                    name_counts_all = None, 
                    colors_gt = None,
                    #names_gt = None,
                    cosine_cluster = False,
                   cosine_color = False,
                    gt_reg_color = False,
                    sel_reg_color = False,
                   n_components = 2,
                   alpha = 1,
                   ptsize = 1,
                    ncord = 6,
                   selected_cosines = False,
                   plot_watch_full = False,
                   plot_watch_results = False,
                   names_gt_plot = None,
                   plot_gt = False,
                   plot_set = True,
                   wheel_font = 100):
    
   
    
    print('loading data')
    with open(result_file,'rb') as inp:
        results = pickle.load(inp, pickle.HIGHEST_PROTOCOL)
        
    atoms4 = results['dictionary']['atoms4']
    dg_M = results['replicates_small'][0].dg_M
    n = dg_M.shape[0]
    m = n_components
    p = results['replicates_small'][0].dg_M.shape[2]
    nreps = results['supports_lasso'][1].shape[0]
    embed = results['embed']
    
    if ground_truth is not None:
        print('compute ground truth values for comparison' )
        cores = pathos.multiprocessing.cpu_count() - 1
        pool = Pool(cores) 
        n = positions.shape[0]
        gt_results = pool.map(lambda i: get_features(positions[i],
                                   atoms2 = ground_truth['atoms2'],
                                   atoms3 = ground_truth['atoms3'],
                                   atoms4 = ground_truth['atoms4']),
            data_stream_custom_range(list(range(n))))
        values_gt = np.vstack([np.hstack(gt_results[i]) for i in range(n)])

    print('getting ground truth names')
    if ground_truth is not None:
        natoms = positions.shape[1]
        superset = results['dictionary']['atoms4']#get_atoms4_full(atoms4) #needs adjustment for diagram dictionaries
        if d == 2:
            j1 = get_index_matching(ground_truth['atoms4'][0], superset) #needs adjustment for non torsion ground truths
            j2 = get_index_matching(ground_truth['atoms4'][1], superset)
            gt_ind = [j1,j2]
            names_gt = get_names(np.asarray(gt_ind))
        if d ==1:
            j1 = get_index_matching(ground_truth['atoms4'][0], superset)
            gt_ind = [j1]
            names_gt = get_names(np.asarray(gt_ind))
            
    if plot_gt:
        print('plotting ground truth function values', names_gt_plot)
        for k in range(len(names_gt_plot)):
            title = r'$g_{{{}}}$'.format(names_gt_plot[k])# + ' ' +  str(atoms4[gt_ind[k]] + 1)
            if n_components == 3:
                plot_manifold_3d(embed, ptsize, alpha, values_gt[:,k], title,title_color = colors_gt[k])#, title_color = colors[s])  
            if n_components == 2:
                plot_manifold_2d(embed, ptsize, alpha, values_gt[:,k], title,title_color = colors_gt[k])
            plt.savefig(outdir + '/gt_' + str(k))
            plt.close()
            
    if ground_truth is None:
        gt_ind = []
        colors_gt = np.empty((0,4), int)
        
    print('cca', color_counts_all)
    if color_counts_all is not None:
        print('getting all colors')
        colors_all = np.empty((0,4), int)
        for c in range(len(color_counts_all)):
            colors_all = np.vstack([colors_all, np.repeat([colors_id_all[c]], color_counts_all[c], axis = 0)])
    
    print('getting colors and names of selected functions')
    selected_lasso = results['selected_lasso']
    colors_lasso = get_cmap(selected_lasso)
    names_lasso = get_names(selected_lasso)
    selected_ts = results['selected_ts']
    colors_ts = get_cmap(selected_ts)
    names_ts = get_names(selected_ts)
    colors_lasso_full = np.zeros((p,4))
    colors_lasso_full[selected_lasso] = colors_lasso
    colors_ts_full = np.zeros((p,4))
    colors_ts_full[selected_ts] = colors_ts

    print('plotting top coordinates in feature space')
    data = results['data']
    title = name
    plot_manifold_featurespace(data,title,ncord)    
    plt.savefig(outdir + '/features')
    plt.close()
    
    print('plotting sample regularization path')
    selection_lambda = np.zeros(nreps)
    prune_lambda = np.zeros(nreps)
    for r in range(nreps):
        rep = results['replicates_small'][r]
        #NOTE (Sam) - min necessary for numerical reasons
        selection_lambda[r] = rep.xaxis_reorder[np.where(np.asarray(np.linalg.norm(rep.cs_reorder, axis = tuple([1,2])), dtype = bool).sum(axis = 1) == d)[0]].min()
        prune_lambda[r] = rep.xaxis_reorder[1]
        
    if gt_reg_color:
        fig, axes_all = plt.subplots(figsize=(15, 15))
        plot_reg_path_ax_lambdasearch_customcolors_norm(axes_all, results['replicates_small'][0].cs_reorder, results['replicates_small'][0].xaxis_reorder * np.sqrt(m*n), fig,colors_all)#axes_all[0].imshow(asdf)
        axes_all.set_xlim(left = 0, right = results['replicates_small'][0].xaxis_reorder.max() * np.sqrt(m*n))
        axes_all.set_title('Regularization path', fontsize = 80)
        axes_all.set_ylabel(r'$||\beta_j||$', fontsize = 100)
        axes_all.set_xlabel(r'$\lambda$', fontsize = 100)
        axes_all.axvline(selection_lambda[0]* np.sqrt(m*n), linewidth = 5, color = 'black')
#         axes_all.text(,x = selection_lambda[0], y = 1)
        plt.tight_layout()
        plt.savefig(outdir + '/reg_path_gt')
        plt.close()
        
    if sel_reg_color:
        fig, axes_all = plt.subplots(figsize=(15, 15))
        plot_reg_path_ax_lambdasearch_customcolors_norm(axes_all, results['replicates_small'][0].cs_reorder , results['replicates_small'][0].xaxis_reorder * np.sqrt(m*n) , fig,colors_lasso_full)#axes_all[0].imshow(asdf)
        axes_all.set_xlim(left = 0, right = results['replicates_small'][0].xaxis_reorder.max() * np.sqrt(m*n))
        axes_all.set_title('Regularization path', fontsize = 80)
        axes_all.set_ylabel(r'$||\beta_j||$', fontsize = 100)
        axes_all.set_xlabel(r'$\lambda$', fontsize = 100)
        axes_all.axvline(selection_lambda[0]* np.sqrt(m*n), linewidth = 5, color = 'black')
        plt.tight_layout()
        plt.savefig(outdir + '/reg_path_sel')
        plt.close()
        
        fig, axes_all = plt.subplots(figsize=(15, 15))
        plot_reg_path_ax_lambdasearch_customcolors_norm(axes_all, results['replicates_small'][0].cs_reorder , results['replicates_small'][0].xaxis_reorder * np.sqrt(m*n) , fig,colors_lasso_full)#axes_all[0].imshow(asdf)
        axes_all.set_xlim(left = 0, right = results['replicates_small'][0].xaxis_reorder.max() * np.sqrt(m*n))
        axes_all.set_title('Regularization path', fontsize = 80)
        axes_all.set_ylabel(r'$||\beta_j||$', fontsize = 100)
        axes_all.set_xlabel(r'$\lambda$', fontsize = 100)
        axes_all.axvline(prune_lambda[0]* np.sqrt(m*n), linewidth = 5, color = 'black')
        plt.tight_layout()
        plt.savefig(outdir + '/reg_path_sel_prune')
        plt.close()
        
    if embedding:
        supports_lasso_values = np.vstack([np.hstack(results['supports_lasso_values'][i]) for i in range(n)])
        supports_ts_values = np.vstack([np.hstack(results['supports_ts_values'][i]) for i in range(n)])
        print('plotting selected function values', colors_ts.shape, supports_ts_values.shape)
        if n_components == 3:
            if plot_set:
                plot_manifold_3d_set(data = embed, s = 10, alpha=.2, gb = supports_lasso_values, titles= results['dictionary']['atoms4'],sub= results['selected_lasso'], title_colors = colors_lasso, title= name)
                plt.savefig(outdir + '/selected_function_lasso_all')
                plt.close()
        for s in range(supports_lasso_values.shape[1]):
            c = supports_lasso_values[:,s]
            title_color = colors_lasso[s]
            title = names_lasso[s] + ' ' + str(atoms4[selected_lasso[s]] + 1)
            if n_components == 3:
                plot_manifold_3d(embed, ptsize, alpha, c, title,title_color = title_color)#, title_color = colors[s])  
            if n_components == 2:
                plot_manifold_2d(embed, ptsize, alpha, c, title,title_color = title_color)#, colors[s])
            plt.savefig(outdir + '/selected_function_lasso_'+ str(s))
            plt.close()
            
        if n_components == 3:
            if plot_set:
                plot_manifold_3d_set(data = embed, s = 10, alpha=.2, gb = supports_ts_values, titles= results['dictionary']['atoms4'],sub= results['selected_ts'], title_colors = colors_ts, title = name)
                plt.savefig(outdir + '/selected_function_ts_all')
                plt.close()
        for s in range(supports_ts_values.shape[1]):
            title_color = colors_ts[s]
            title = names_ts[s] + ' ' + str(atoms4[selected_ts[s]] + 1)
            c = supports_ts_values[:,s]
            if n_components == 3:
                plot_manifold_3d(embed, ptsize, alpha, c, title,title_color = title_color)#, colors_ts[s])  
            if n_components == 2:
                plot_manifold_2d(embed, ptsize, alpha, c, title,title_color = title_color)#, colors_ts[s])   
            plt.savefig(outdir + '/selected_function_ts_' + str(s))
            plt.close()

    if name_counts_all is None:
        names_all = list(range(p))
    else:
        names_all = np.asarray(list([]), dtype =str)
        for i in range(len(name_counts_all)):
            for j in range(name_counts_all[i]):
                names_all = np.hstack([names_all,(str(i+1) + ',' + str(j+1))])     
               
    print("plotting watches")
    if plot_watch_full:
        p = results['supports_lasso'][0].shape[0]
        fig, axes_all = plt.subplots(figsize=(15, 15))
        plot_watch_custom(results['supports_lasso'][0], p, axes_all,colors_all, nreps, names_all, fontsize= wheel_font)
        axes_all.set_title('Estimated Support', fontsize = 80, pad = 65)
        plt.savefig(outdir + '/watch_full')
        plt.close()
        
        fig, axes_all = plt.subplots(5,4,figsize = (45,45))
        plot_reg_path_ax_lambdasearch_customcolors(axes_all[0], results['replicates_small'][0].cs_reorder, results['replicates_small'][0].xaxis_reorder * np.sqrt(m * n), fig,colors_all,names_all)
        plot_reg_path_ax_lambdasearch_customcolors(axes_all[1], results['replicates_small'][1].cs_reorder, results['replicates_small'][1].xaxis_reorder * np.sqrt(m * n), fig,colors_all,names_all)
        plot_reg_path_ax_lambdasearch_customcolors(axes_all[2], results['replicates_small'][2].cs_reorder, results['replicates_small'][2].xaxis_reorder * np.sqrt(m * n), fig,colors_all,names_all)
        plot_reg_path_ax_lambdasearch_customcolors(axes_all[3], results['replicates_small'][3].cs_reorder, results['replicates_small'][3].xaxis_reorder * np.sqrt(m * n), fig,colors_all,names_all)
        plot_reg_path_ax_lambdasearch_customcolors(axes_all[4], results['replicates_small'][4].cs_reorder, results['replicates_small'][4].xaxis_reorder * np.sqrt(m * n), fig,colors_all,names_all)
        for i in range(4):
            for j in range(4):
                axes_all[i][j].set_xlabel('')
        for i in range(4):
            for j in range(4):
                axes_all[i+1][j].set_title('')       
        for i in range(5):
            for j in range(3):
                axes_all[i][j+1].set_ylabel('')        
        axes_all[0,0].set_ylabel('Replicate 1 \n' + r'$\|\beta_j\|$', fontsize = 100)
        axes_all[1,0].set_ylabel('Replicate 2 \n' + r'$\|\beta_j\|$', fontsize = 100)
        axes_all[2,0].set_ylabel('Replicate 3 \n' + r'$\|\beta_j\|$', fontsize = 100)
        axes_all[3,0].set_ylabel('Replicate 4 \n' + r'$\|\beta_j\|$', fontsize = 100)
        axes_all[4,0].set_ylabel('Replicate 5 \n' + r'$\|\beta_j\|$', fontsize = 100)
        fig = plt.gcf()
        fig.subplots_adjust(top=0.9)
        fig.suptitle(name,y=.95,x = .53, fontsize=80)
        plt.savefig(outdir + '/first5reps_cords')
        
        
    if plot_watch_results:
        sub = results['supports_lasso'][0]
        for w in range(len(results['supports_lasso'][0].shape)):
            sub = np.take(sub, selected_lasso, axis = w)
        fig, axes_all = plt.subplots(figsize=(15, 15))
        plot_watch(sub, names=names_lasso, ax = axes_all,colors = colors_lasso, nreps = nreps) 
        #axes_all.text(0,1.5,'Estimated Support', fontsize = 60)
        axes_all.set_title('Estimated Support', fontsize = 80, pad = 65)
        #plt.tight_layout()
        plt.savefig(outdir + '/watch_lasso')
        plt.close()
        
        sub = results['supports_ts'][0]
        for w in range(len(results['supports_ts'][0].shape)):
            sub = np.take(sub, selected_ts, axis = w)      
        fig, axes_all = plt.subplots(figsize=(15, 15))
        plot_watch(sub, names=names_ts, ax = axes_all,colors = colors_ts, nreps = nreps) 
        axes_all.set_title('Estimated Support', fontsize = 80, pad = 65)
        #plt.tight_layout()
        plt.savefig(outdir + '/watch_ts')
        plt.close()
        
    if d > 1:
        if cosine_color:
            cosines_full= results['replicates_small'][0].cosine_abs
            print("plotting full cosine matrix colored",cosines_full.shape)
            fig, axes_all = plt.subplots(figsize=(15, 15))
            plot_cosines(cosines_full, axes_all, colors_all, names_all) 
            plt.tight_layout()
            plt.savefig(outdir + '/cosine_colored')
            plt.close()
        
        if cosine_cluster:
            print("plotting full cosine matrix clustered")
            cosines_full= results['replicates_small'][0].cosine_abs
            plot_cosines_cluster(cosines_full)
            plt.savefig(outdir + '/cosines_clustered')
            plt.close()
            
        if selected_cosines:
            print("plotting cosines of ground truth and selected lasso")
            selected_lasso_gt = np.concatenate((gt_ind, selected_lasso)) #add 234
            colors_lasso_plusgt = np.vstack([colors_gt, colors_lasso])
            names_lasso_plusgt = np.concatenate((names_gt, names_lasso))
            cuz_l = np.abs(get_cosines(results['replicates_small'][0].dg_M[:,:,selected_lasso_gt]))
            cuz_l0 = np.mean(cuz_l, axis = 0)
            fig, axarr = plt.subplots(figsize=(15, 15))
            plot_cos_boxes(results['supports_lasso'][1], names_lasso_plusgt, cuz_l0, selected_lasso_gt, d , nreps,axarr)
            for xtick, color in zip(axarr.get_xticklabels(), colors_lasso_plusgt):
                xtick.set_color(color)
            for ytick, color in zip(axarr.get_yticklabels(), colors_lasso_plusgt):
                ytick.set_color(color)
            axarr.set_title(r"$\frac{1}{n'} \sum_{i = 1}^{n'} \frac{|\langle grad_{\mathcal M} g_j (\xi_i) ,grad_{\mathcal M} g_{j'} (\xi_i)\rangle|}{\|grad_{\mathcal M} g_j (\xi_i) \| \| grad_{\mathcal M} g_{j'}(\xi_i) \|} $" ,
                            fontsize = 80,pad = 50)
            plt.tight_layout()
            plt.yticks(rotation= 0)
            plt.savefig(outdir + '/cosines_sellasso_gt')
            plt.close()
            
            print("plotting cosines of ground truth and selected ts")
            selected_ts_gt = np.concatenate((gt_ind, selected_ts)) #add 234
            colors_ts_plusgt = np.vstack([colors_gt, colors_ts])
            names_ts_plusgt = np.concatenate((names_gt, names_ts))
            cuz_l = np.abs(get_cosines(results['replicates_small'][0].dg_M[:,:,selected_ts_gt]))
            cuz_l0 = np.mean(cuz_l, axis = 0)
            fig, axarr = plt.subplots(figsize=(15, 15))
            plot_cos_boxes(results['supports_ts'][1], names_ts_plusgt, cuz_l0, selected_ts_gt, d , nreps,axarr)
            for xtick, color in zip(axarr.get_xticklabels(), colors_ts_plusgt):
                xtick.set_color(color)
            for ytick, color in zip(axarr.get_yticklabels(), colors_ts_plusgt):
                ytick.set_color(color)
            axarr.set_title(r"$\frac{1}{n'} \sum_{i = 1}^{n'} \frac{|\langle grad_{\mathcal M} g_j (\xi_i) ,grad_{\mathcal M} g_{j'} (\xi_i)\rangle|}{\|grad_{\mathcal M} g_j (\xi_i) \| \| grad_{\mathcal M} g_{j'}(\xi_i) \|} $" ,
                            fontsize = 80,pad = 50)
            plt.tight_layout()
            plt.yticks(rotation= 0)   
            plt.savefig(outdir + '/cosines_selts_gt')
            plt.close()
            
    if not plot_watch_full:
        
        print("getting correlations with ground truth")
        coses = np.zeros((nreps,d,d))
        for r in range(nreps):
            print(r)
            j3 = int(results['supports_lasso'][1][r,0])
            j4 = int(results['supports_lasso'][1][r,1])
            
            rep = results['replicates_small'][0]
            nsel = rep.dg_M.shape[0]
            coses[r,0,0] = np.sum(np.abs(np.asarray([cosine_similarity(rep.dg_M[i,:,j1], rep.dg_M[i,:,j3]) for i in range(nsel)]))) / nsel
            coses[r,0,1] = np.sum(np.abs(np.asarray([cosine_similarity(rep.dg_M[i,:,j1], rep.dg_M[i,:,j4]) for i in range(nsel)]))) / nsel
            coses[r,1,0] = np.sum(np.abs(np.asarray([cosine_similarity(rep.dg_M[i,:,j2], rep.dg_M[i,:,j3]) for i in range(nsel)]))) / nsel
            coses[r,1,1] = np.sum(np.abs(np.asarray([cosine_similarity(rep.dg_M[i,:,j2], rep.dg_M[i,:,j4]) for i in range(nsel)]))) / nsel
            #pd.save('save')
            
        coses_i = np.zeros((nreps,2))
        for r in range(nreps):
            mean1 = np.mean([coses[r, 0,1], coses[r, 1,0]])
            mean2 = np.mean([coses[r, 0,0], coses[r, 1,1]])
            betterpair = np.argmax([ mean1,mean2])
            if betterpair == 0:
                coses_i[r] = [coses[r, 0,1], coses[r, 1,0]]
            else:
                coses_i[r] = [coses[r, 1,1], coses[r, 0,0]]
                
        print('lasso matched cos mean',np.mean(coses_i))
        print('lasso matched cos var', np.var(coses_i)**(0.5))
        print("getting correlations with ground truth")
        coses = np.zeros((nreps,d,d))
        
        for r in range(nreps):
            print(r)
            j3 = results['supports_ts'][1][r,0]
            j4 = results['supports_ts'][1][r,1]
            
            rep = results['replicates_small'][0]
            nsel = rep.dg_M.shape[0]
            coses[r,0,0] = np.sum(np.abs(np.asarray([cosine_similarity(rep.dg_M[i,:,j1], rep.dg_M[i,:,j3]) for i in range(nsel)]))) / nsel
            coses[r,0,1] = np.sum(np.abs(np.asarray([cosine_similarity(rep.dg_M[i,:,j1], rep.dg_M[i,:,j4]) for i in range(nsel)]))) / nsel
            coses[r,1,0] = np.sum(np.abs(np.asarray([cosine_similarity(rep.dg_M[i,:,j2], rep.dg_M[i,:,j3]) for i in range(nsel)]))) / nsel
            coses[r,1,1] = np.sum(np.abs(np.asarray([cosine_similarity(rep.dg_M[i,:,j2], rep.dg_M[i,:,j4]) for i in range(nsel)]))) / nsel
            
        coses_i = np.zeros((nreps,2))
        for r in range(nreps):
            mean1 = np.mean([coses[r, 0,1], coses[r, 1,0]])
            mean2 = np.mean([coses[r, 0,0], coses[r, 1,1]])
            betterpair = np.argmax([ mean1,mean2])
            if betterpair == 0:
                coses_i[r] = [coses[r, 0,1], coses[r, 1,0]]
            else:
                coses_i[r] = [coses[r, 1,1], coses[r, 0,0]]
                
        print('ts matched cos mean',np.mean(coses_i))
        print('ts matched cos var', np.var(coses_i)**(0.5))
        
    min_mins = np.zeros(nreps, dtype = np.float64)
    mus = np.zeros(nreps, dtype = np.float64)
    kappas = np.zeros(nreps, dtype = np.float64)
    gamma_maxs = np.zeros(nreps, dtype = np.float64)
    for r in range(nreps):
        dg_M = results['replicates_small'][r].dg_M
        min_mins[r]  = get_min_min(dg_M[:,:,gt_ind])
        mus[r] = get_mu_full_ind(np.swapaxes(dg_M,1,2), gt_ind)
        kappas[r] = get_kappa_s(dg_M[:,:,gt_ind])
        gamma_maxs[r] = get_gamma_max(dg_M[:,:,gt_ind])

    print("min min mean, std", np.mean(min_mins), np.var(min_mins)**0.5)
    print("mu mean, std", np.mean(mus), np.var(mus)**0.5)
    print("kappa mean, std", np.mean(kappas), np.var(kappas)**0.5)
    print("gamma_max mean, std", np.mean(gamma_maxs), np.var(gamma_maxs)**0.5)

In [4]:
import matplotlib.pyplot as plt
import numpy as np